### preprocess_pm.ipynb – Datenaufbereitung für die Frequenzanalyse

Dieses Notebook bereitet die Inhalte der BVG-Pressemitteilungen strukturiert auf.  
Der Fokus liegt auf der Extraktion des eigentlichen Pressetexts und der anschließenden Bereinigung.  
Die Ergebnisse (Wortfrequenzen) dienen als Grundlage für nachgelagerte Analysen.

#### 1. Import benötigte Pakete

In [3]:
# Standard
import os
import pandas as pd # Datenanalyse
import sqlite3  # Speicherung in SQLite-Datenbanken
import sys  # Systemfunktionen 

# Bearbeiten von HTML-Dateien
from bs4 import BeautifulSoup  # HTML auslesen und bereinigen
from collections import Counter

# Eigene Funktionen (ausgelagert)
sys.path.append("../scripts") # Pfad zu den Funktionen
# Funktionen aus datenaufbereitung.py importieren
from datenaufbereitung import (
    load_stopwords,
    read_html_file,
    extract_date_from_html,
)
stopwords_list = load_stopwords()

In [4]:
# Pfade
INPUT_FOLDER = os.path.join("..", "input", "pm_bvg_raw")
OUTPUT_FOLDER = os.path.join("..", "output")

In [5]:
# CSV Datei Wortzählung einlesen
df = pd.read_csv("output/pm_bvg_wordcount.csv")

In [6]:
# Top 10 Wörter BVG-Webseite
top10 = df.groupby("word")["count"].sum().sort_values(ascending=False).head(10)
print(top10)

word
bvg                     6693
berliner                1497
service                 1424
startseite              1397
pressemitteilungen      1395
cookie-einstellungen    1395
werden.                 1191
informationen           1020
2025                     992
tickets                  946
Name: count, dtype: int64


In [7]:
# DEBUG Wird der bereinigte DataFrame wirklich gespeichert?
print(df.head(10))

   count                  word  source        date
0     12                   bvg  bvg_pm  2021-08-06
1      8              biesdorf  bvg_pm  2021-08-06
2      6              berliner  bvg_pm  2021-08-06
3      4                berlin  bvg_pm  2021-08-06
4      4                    13  bvg_pm  2021-08-06
5      3               werden.  bvg_pm  2021-08-06
6      3                points  bvg_pm  2021-08-06
7      3               service  bvg_pm  2021-08-06
8      3  cookie-einstellungen  bvg_pm  2021-08-06
9      3                    go  bvg_pm  2021-08-06


#### 2. Definition der Verarbeitungsfunktionen (Extraktion & Bereinigung)  

In [8]:
# Eigentlicher Pressetext in Wörter umwandeln, Stoppwörter entfernen
def clean_and_split_text(text, stopwords_list):
    # Text in Kleinbuchstaben + Zeilenumbrüche raus
    text = text.lower().replace("\n", " ")

    # In Tokens aufteilen
    tokens = text.split()

    # Stoppwörter und kurze Wörter raus
    return [w for w in tokens if len(w) > 1 and w not in stopwords_list]

In [9]:
def extract_press_text(html):
    soup = BeautifulSoup(html, "html.parser")

    # Aktuelle Pressemitteilungen (mit <p>-Tags)
    paragraphs = soup.find_all("p")
    if paragraphs:
        return " ".join(p.get_text() for p in paragraphs)

    # Ältere Pressemitteilungen (RichText-Klasse)
    richtext_divs = soup.find_all("div", class_="RichText_RichText__F_qRr")
    if richtext_divs:
        return " ".join(div.get_text() for div in richtext_divs)

    # 3. Fallback
    return soup.get_text(separator=" ")

#### 3. Anwendung der Verarbeitungsfunktionen auf die Pressemitteilungen  

In [10]:
# Ergebnisse sammeln
collection = []

# Pressemitteilungen (HTML-Dateien) zur Verarbeitung laden
files = [f for f in os.listdir(INPUT_FOLDER) if f.endswith(".html")]

for file in files:
    file_path = os.path.join(INPUT_FOLDER, file)
    try:
        # HTML laden mit ausgelagerter Funktion
        html = read_html_file(file_path)

        # Datum extrahieren mit ausgelagerter Funktion
        date_str, year = extract_date_from_html(html)

        # Nur Pressetext extrahieren
        text = extract_press_text(html)

        # Text bereinigen und in Wörter umwandeln
        words = clean_and_split_text(text, stopwords_list)

        # Häufigkeit jedes Wortes zählen mit Counter
        word_counts = Counter(words)

        # Als DataFrame formatieren mit Spalten Wort, Medium, Datum
        count_df = pd.DataFrame(word_counts.items(), columns=["word", "count"])
        count_df["source"] = "bvg_pm"
        count_df["date"] = date_str

        # Ergebnis speichern
        collection.append(count_df)

    except Exception as e:
        print(f"Fehler bei Datei {file}: {e}")

In [11]:
# Alle Ergebnis speichern, in DataFrame umwandeln
df_all = pd.concat(collection, ignore_index=True)


In [12]:
# Ausgewählte Jahre filtern

# Datum als Datetime
df_all["date"] = pd.to_datetime(df_all["date"])

# Nur Daten von 2021 bis 2025 behalten
df_all = df_all[df_all["date"].dt.year.between(2021, 2025)]

# Jahrsspalte hinzufügen
df_all["year"] = df_all["date"].dt.year

In [30]:
df_all["year"].value_counts().sort_index()

year
2021    13934
2022    15172
2023    10863
2024     9257
2025     2806
Name: count, dtype: int64

In [13]:
# DEBUG  1. Wurden Stoppwörter überhaupt geladen?
print("Beispiel-Stoppwörter:", stopwords_list[:10])
print("Anzahl Stoppwörter:", len(stopwords_list))

Beispiel-Stoppwörter: ['ab', 'aber', 'abermaliges', 'abermals', 'abgerufen', 'abgerufene', 'abgerufener', 'abgerufenes', 'abgesehen', 'acht']
Anzahl Stoppwörter: 1854


In [14]:
# DEBUG 2. Wird clean_and_split_text() überhaupt aufgerufen?
print("[DEBUG] Originaltext (100 Zeichen):", text[:100])
print("[DEBUG] Erste 10 Tokens vor Bereinigung:", text.lower().replace("\n", " ").split()[:10])
print("[DEBUG] Erste 10 Wörter nach Bereinigung:", words[:10])

[DEBUG] Originaltext (100 Zeichen): Pressemitteilung, 26.07.2018. Am U-Bahnhof Zwickauer Damm wird in den kommenden Monaten einiges los 
[DEBUG] Erste 10 Tokens vor Bereinigung: ['pressemitteilung,', '26.07.2018.', 'am', 'u-bahnhof', 'zwickauer', 'damm', 'wird', 'in', 'den', 'kommenden']
[DEBUG] Erste 10 Wörter nach Bereinigung: ['pressemitteilung,', '26.07.2018.', 'u-bahnhof', 'zwickauer', 'damm', 'kommenden', 'monaten', 'los', 'sorgen', 'ausnahmsweise']


In [16]:
# DEBUG 3. Wird der bereinigte DataFrame wirklich gespeichert?
print(df_all.head(10))

                      word  count  source       date  year
108  …marzahn-hellersdorf!      1  bvg_pm 2021-08-06  2021
109                 punkte      1  bvg_pm 2021-08-06  2021
110         berliner*innen      1  bvg_pm 2021-08-06  2021
111                 bezirk      2  bvg_pm 2021-08-06  2021
112           musikalische      1  bvg_pm 2021-08-06  2021
113            darbietung,      1  bvg_pm 2021-08-06  2021
114               modernes      1  bvg_pm 2021-08-06  2021
115     mobilitätsangebot.      1  bvg_pm 2021-08-06  2021
116               freitag,      1  bvg_pm 2021-08-06  2021
117                     6.      1  bvg_pm 2021-08-06  2021


In [17]:
print("[DEBUG] Spaltennamen:", df_all.columns)
print("[DEBUG] Beispiel-Wörter:", df_all["word"].unique()[:10])

[DEBUG] Spaltennamen: Index(['word', 'count', 'source', 'date', 'year'], dtype='object')
[DEBUG] Beispiel-Wörter: ['…marzahn-hellersdorf!' 'punkte' 'berliner*innen' 'bezirk' 'musikalische'
 'darbietung,' 'modernes' 'mobilitätsangebot.' 'freitag,' '6.']


In [18]:
# Prüfung
print(df_all.shape)
print(f"Verarbeitet: {len(files)} Dateien")
print(f"Gesamtzahl Wörter: {len(df)}")

(52032, 5)
Verarbeitet: 910 Dateien
Gesamtzahl Wörter: 143198


In [19]:
# Prüfung
print(df_all.shape)
print(f"Verarbeitet: {len(files)} Dateien")
print(f"Gesamtzahl Wörter: {len(df)}")

(52032, 5)
Verarbeitet: 910 Dateien
Gesamtzahl Wörter: 143198


In [21]:
# Nur Pressetext extrahieren
text = extract_press_text(html)

# Text bereinigen
words = clean_and_split_text(text, stopwords_list)
print(f"[DEBUG] Erste 10 Wörter im Originaltext: {text.lower().split()[:10]}")
print(f"[DEBUG] Erste 10 Wörter nach Bereinigung: {words[:10]}")

[DEBUG] Erste 10 Wörter im Originaltext: ['pressemitteilung,', '26.07.2018.', 'am', 'u-bahnhof', 'zwickauer', 'damm', 'wird', 'in', 'den', 'kommenden']
[DEBUG] Erste 10 Wörter nach Bereinigung: ['pressemitteilung,', '26.07.2018.', 'u-bahnhof', 'zwickauer', 'damm', 'kommenden', 'monaten', 'los', 'sorgen', 'ausnahmsweise']


#### 5. Speicherung der Ergebnisse (Wortfrequenzen)

In [22]:
# Als csv speichern
CSV_CLEAN_PATH = os.path.join(OUTPUT_FOLDER, "wordcount_pm_clean.csv")
df_all.to_csv(CSV_CLEAN_PATH, index=False)

print(f"Bereinigte Wortzählung als CSV gespeichert unter: {CSV_CLEAN_PATH}")

Bereinigte Wortzählung als CSV gespeichert unter: ..\output\wordcount_pm_clean.csv


In [23]:
# Prüfung: Vergleich alte und neue CSV-Datei
# Alte Datei
df_old = pd.read_csv("output/pm_bvg_wordcount.csv")  

# Neue bereinigte Datei
df_new = pd.read_csv("output/wordcount_pm_clean.csv")

In [24]:
# Wortfrequenz berechnen (gesamt)
top_old = df_old.groupby("word")["count"].sum().sort_values(ascending=False)
top_new = df_new.groupby("word")["count"].sum().sort_values(ascending=False)

In [25]:
set(top_old.index) - set(top_new.index)

{'05.05.2022',
 '…war',
 'einvernehmen',
 'bvg-warnstreik',
 'tri,',
 'fahrgäste.“',
 '6x',
 'dach-region.',
 'ordnungsämter.“',
 'zukommt.“',
 'ableger',
 'auguste-viktoria-krankenhauses.',
 'ansteigen.',
 'modetestgruppe',
 '05.04.2023',
 '448',
 'streich',
 'bahnen!',
 '50-meter-straßenbahnen',
 'u-bahnfahrzeuge.',
 '16.07.2021',
 'www.jelbi.de/lindenhof',
 'profitiert,',
 'konnten.“',
 'marschall',
 'tauschgeschäft',
 'denker',
 '„bronze“.',
 'ohrwurmalarm!',
 'viertels',
 'ausüben.',
 '21.000',
 'idealfall',
 'vor-ab',
 'belastung.',
 'tickets.',
 'wirtschaftlicher',
 'impf-shuttle',
 'marzahn-hellersdorf.',
 '05.12.2022',
 'zuschläge.',
 'recruiting-kampagne,',
 'recruiting“).',
 '15.10.2021',
 'gebaut…',
 'www.jelbi.de/biesdorf',
 'krankmeldungen.',
 'gleisberechtigung',
 'haltestellen.”',
 'ingenieurwesen,',
 '\t\xa0',
 'fragezeiten',
 'fahrer*innen.\xa0',
 'emmaus-kirchengemeinde',
 '16.03.2021',
 'was!',
 'stillgelegten',
 'wäre.“',
 'bedingten,',
 'ort.“',
 'übernahme',
 'zu

In [26]:
set(top_new.index) - set(top_old.index)

{'25.05.2018.',
 'philippe',
 'u-bahn.nötig',
 'kooperativen',
 'digitalisierung,',
 'genannt):',
 'ausbauen.während',
 'www.berlkönig.de.\xa0kontaktberliner',
 'aus.zum',
 'jahre.neben',
 'routenkonzept',
 'akkugestresste',
 'schillingstraße',
 'metrobus-linien',
 'und\xa0menschen,',
 'eindeutig',
 'stromnetz.die',
 'updates',
 '17.05.2018.\xa0ganz',
 'u-bahn-strecken',
 'abgegrenzt',
 'wurde.\xa0das',
 'e-mail-adresse',
 '-tretroller.',
 'lte-versorgung',
 'gearbeitet.der',
 'gewandfür',
 'aufgehoben.wie',
 'geräte-,',
 'keh.während',
 'akribisch',
 'durfte.',
 'das\xa0achte',
 'verkehrsvertrages.dr.',
 'europa.mehr',
 'i/4',
 'komplett.“',
 'myfest',
 'meile“.',
 '61/63',
 'nationen.alle',
 'kreativleistung',
 '25.11.2019.',
 '170er',
 '10-minuten-mindesttakte',
 '\u200bpressemitteilung,',
 'november:',
 'statik',
 'buntgeschmückte',
 'sneakers',
 '-verbünde',
 'stimmte',
 'kauft',
 'verkehrslenkung',
 'kunst.',
 'hin.',
 'kernwerte',
 'tankstelle.',
 'mobilität.“„wir',
 'wbm',
 'kl

In [27]:
# Pfade
INPUT_FOLDER = os.path.join("..", "input", "pm_bvg_raw")
OUTPUT_FOLDER = os.path.join("..", "output")

In [28]:
# Als SQLite-Datenbank speichern
SQL_CLEAN_PATH = os.path.join(OUTPUT_FOLDER, "dwh_pm_bvg.sqlite3")
conn = sqlite3.connect(SQL_CLEAN_PATH)
df_all.to_sql("wordcount_pm_clean", conn, if_exists="replace", index=False)
conn.close()

print(f"Bereinigte Wortzählung gespeichert unter: {SQL_CLEAN_PATH}")

Bereinigte Wortzählung gespeichert unter: ..\output\dwh_pm_bvg.sqlite3


In [29]:
# Prüfung Tabellen in Datenbank
# Verbindung zur Datenbank aufbauen
conn = sqlite3.connect(os.path.join(OUTPUT_FOLDER, "dwh_pm_bvg.sqlite3"))

# Cursor-Objekt erstellen
cursor = conn.cursor()

# Abfrage: Alle Tabellennamen aus der Datenbank holen
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")

# Ergebnis holen
tables = cursor.fetchall()

# Tabellen ausgeben
print("Tabellen in der Datenbank:")
for table in tables:
    print(table[0])

# Verbindung schließen
conn.close()

Tabellen in der Datenbank:
pm_bvg_valid
wordcount_pm
wordcount_pm_clean
